In [3]:
# import potřebných modulů
import pandas as pd
import datapackage
import wbgapi as wb

## Definice funkcí, které organizují statistická data z různých zdrojů do tabulky.

In [5]:
def iso_kody():
    '''Stáhnout iso kódy a kontinentální příslušnost všech zemí světa.'''
    # iso kódy a názvy států
    data_url = 'https://datahub.io/core/country-codes/datapackage.json'
    package = datapackage.Package(data_url)
    zdroje = package.resources
    for zdroj in zdroje:
        if zdroj.tabular:
            kody = pd.read_csv(zdroj.descriptor['path'])
    iso_dict = dict(zip(kody['ISO3166-1-Alpha-3'],
                        kody['UNTERM English Short']))
    iso_df = pd.DataFrame.from_dict(iso_dict,
                                    orient='index',
                                    columns=['Stat'])
    iso_df.index.names=['iso']
    # info o kontinentu
    kontinent_dict = dict(zip(kody['ISO3166-1-Alpha-3'],
                              kody['Region Name']))
    kontinent_df = pd.DataFrame.from_dict(kontinent_dict,
                                          orient='index',
                                          columns=['Region'])
    return iso_df, kontinent_df

def les():
    '''
    Otevřít data z Global Forest Watch ohledně celkové spálené
    plochy lesů pro každou zemi.
    '''
    # spálená plocha po týdnech, takže sečíst abychom dostali data za rok
    df = pd.read_csv('..\\..\\resource\\GFW_Global_regions_with_the_most_burned_area\\modis_burned_area__ha.csv')
    iso = df.iso.unique()  # iso kody
    df = df.groupby('iso').sum()  # sečíst data náležící jednotlivým zemím (řádky tabulky se stejným iso kódem)
    df.drop(columns=['alert__year', 'alert__week'], inplace=True)  # vyřadit nepotřebné sloupce
    df.rename(columns={'burned_area__ha': 'spal_plocha_ha'}, inplace=True)
    
    # přidáme celkovou plochu lesa pro každou zemi
    plocha_lesa = pd.read_csv('..\\..\\resource\\GFW_Global_regions_with_the_most_burned_area\\treecover_extent_in__by_region__ha.csv')
    
    df['plocha_lesa_ha'] = 0
    for i in iso:
        df.loc[i, 'plocha_lesa_ha'] = float(plocha_lesa[plocha_lesa['iso']==i].area__ha)
    return df

def zmena_t():
    '''
    Změna teploty v porovnání s periodou mezi lety 1951-1980
    Zde průměruji teploty mezi lety 2012-2020 k získání jedné hodnoty pro každou zem
    '''
    zmena_t = pd.read_csv('..\\..\\resource\\t_change\\Environment_Temperature_change_E_All_Data.csv')
    dt = zmena_t.groupby(by=['Area', 'Element']).mean()
    dt = dt.drop(columns=['Area Code', 'Months Code', 'Element Code'])
    dt = dt[['Y2012', 'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019', 'Y2020']]
    dt = dt.reset_index()
    dt = dt.set_index('Area')
    dt = dt[dt['Element']=='Temperature change'].mean(axis=1)
    dt_df = dt.to_frame(name='zmena_t')
    dt_df.index.names=['Stat']
    return dt_df

def srazky():
    '''
    Srážková data v mm/rok pro každý stát.
    '''
    srazky = pd.read_csv('..\\..\\resource\\average-precipitation-per-year.csv')
    srazky = srazky.groupby('Code').mean()
    srazky = srazky.drop(columns=['Year'])
    srazky.columns = ['prum_srazky_mm_rok']
    return srazky


def spojit_dfs(dfs, on='iso'):
    '''
    Sjednocení dat států z pandas Dataframes,
    které jsou indexovány podle iso kódů.
    '''
    total = dfs[0].join(dfs[1], on=on)
    for i in range(2,len(dfs)):
        total = total.join(dfs[i], on=on)
    return total

## Vygenerování tabulek (pandas DataFrames) 

In [6]:
iso_df, kontinent_df = iso_kody()

In [7]:
lesy = les()
dt = zmena_t()
dt = iso_df.join(dt, on='Stat')
dt = dt.drop(columns=['Stat'])
srazky = srazky()

## Sjednocení všech DataFramů (tabulek) do jedné

In [8]:
df = spojit_dfs((lesy, iso_df, kontinent_df, dt, srazky), on='iso')

In [9]:
df.head()

,spal_plocha_ha,plocha_lesa_ha,Stat,Region,zmena_t,prum_srazky_mm_rok
iso,,,,,,
AFG,3.737909e+03,5.150857e+08,Afghanistan,Asia,1.107725,327.0
AGO,8.027034e+05,9.979384e+08,Angola,Africa,1.151922,1010.0
ALB,5.537902e+02,2.298830e+07,Albania,Europe,1.646248,1485.0
ARE,3.873991e+02,5.696358e+07,United Arab Emirates (the),Asia,NaN,78.0
ARG,1.283496e+06,2.224081e+09,Argentina,Americas,0.754660,591.0


In [10]:
# Data nejsou kompletní, ze 195 států máme
len(df)

148

In [11]:
# Po vyloučení států, kde chybí jakýkoli údaj (naznačeno jako NaN v tabulce)
df = df.dropna()

In [12]:
# Zbyde nám jen asi 60% států světa
len(df)

119

## Kosmetické změny a uložení tabulky

In [13]:
# zaokrouhlení číselných dat
df = df.round(2)

In [14]:
# změna pořadí sloupců
df = df[['Stat', 'Region', 'zmena_t', 'prum_srazky_mm_rok', 'plocha_lesa_ha', 'spal_plocha_ha']]

In [15]:
# uložení dat ve formátu .csv
df.to_csv('data_pozary.csv')

In [16]:
# náhled prvních 5 řádků
df.head()

,Stat,Region,zmena_t,prum_srazky_mm_rok,plocha_lesa_ha,spal_plocha_ha
iso,,,,,,
AFG,Afghanistan,Asia,1.11,327.0,5.150857e+08,3737.91
AGO,Angola,Africa,1.15,1010.0,9.979384e+08,802703.37
ALB,Albania,Europe,1.65,1485.0,2.298830e+07,553.79
ARG,Argentina,Americas,0.75,591.0,2.224081e+09,1283496.22
ARM,Armenia,Asia,1.61,562.0,2.375207e+07,1776.87
